<a href="https://colab.research.google.com/github/neurologic/Neurobiology-Movement/blob/main/executable/NotebookColab_CellTypes_InquiryAndAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cell Types

## The brain has thousands of different types of cells. 

Critically, different cell types serve different functions because they can perform different computations and have unique effects on circuit dynamics - why?. What makes them different? How do we even begin to tease them apart?

![](https://canvas.brown.edu/courses/851434/files/38768331/preview?verifier=D6ZNKahSL6e9e6xh4GMAfbbSJK6ynSYYQwmcKBo8)

We can define neurons by their <b>gene expression patterns</b>, <b>electrophysiology features</b>, and <b>structure</b>. Here, we'll use those three features to compare and contrast cell types in the brain.

This notebook will help us investigate specific features in the electrophysiology dataset from the Allen Brain Atlas. 

**Before diving into this notebook:** Complete "Cell Types Neurophysiology - Part 1", which asks you to look at this data on the Allen Institute website. 


# Set up our coding environment

In [ ]:
#@markdown <b>TASK: </b> RUN this cell to mount your Google Drive, where you will access the raw data from.
#@markdown > Follow all instructions as prompted by pop-ups.
from google.colab import drive
drive.mount('/content/drive')




The cells below will install packages into your coding environment -- we are *not* installing anything on your computer.

The Allen Institute has compiled a set of code and tools called a **Software Development Kit** (SDK). These tools will help us import and analyze the cell types data at the Allen Institute database. 

After you run the cell below, click the **Restart Runtime** button that pops up, and then you're ready to proceed. 

In [ ]:
#@markdown **TASK:** Run this cell to setup the notebook coding environment
#@markdown . If you are prompted to **Restart Runtime** 
#@markdown when it is finished, do so before proceeding. 
# This will ensure that the AllenSDK is installed.
try:
    import allensdk
    print('allensdk imported')
except ImportError as e:
    !pip install allensdk


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
from scipy import signal
import ipywidgets as widgets

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

manifest_file = '/content/drive/Shareddrives/BIOL358/Data/cell_types/manifest.json'

#Import the "Cell Types Cache" from the AllenSDK core package
from allensdk.core.cell_types_cache import CellTypesCache

#Initialize the cache as 'ctc' (cell types cache)
ctc = CellTypesCache(manifest_file)

print('CellTypesCache imported.')

<i>Note</i>: At some points throughout this notebook, you may get an `H5pyDeprecationWarning`, but don't worry about it. This is out of our control. :)

# Examine Data Across Many Cells


## Analyze pre-computed features across Many Cells.

The Cell Types Database contains a set of electrophysiology features that have already been computed for each cell. This serves as good starting points for analysis. We can query the database to get these features.

[Here's a glossary](https://drive.google.com/file/d/1ImWFZ5CvBErp7abHAqJlw9VvNJqyVgjd/view?usp=sharing).

![](https://github.com/ajuavinett/CellTypesLesson/blob/master/docs/ap_features.png?raw=true)
<br>Image from the <a href="http://help.brain-map.org/download/attachments/8323525/CellTypes_Ephys_Overview.pdf">Allen Institute Cell Types Database Technical Whitepaper.</a>
<br><br>

## Compare Features.


In [1]:
#@markdown **TASK:** Run this cell to create a queriable dataframe of 
#@markdown all the measured and calculated intrinsic features.
#@markdown > NOTE: It may take a minute or two to run.

#@markdown A dropdown menu will be created from which you can select parameters.
#@markdown Once you create the dropdown menu, you will not need to re-run this cell to change your selections.
# No need to edit below this line
################################

# Download all electrophysiology features for all cells
ephys_features = ctc.get_ephys_features()
dataframe = pd.DataFrame(ephys_features).set_index('specimen_id')

# Get information about our cells' dendrites
cells = ctc.get_cells()
full_dataframe = dataframe.join(pd.DataFrame(cells).set_index('id'))

print('Ephys features available for %d cells:' % len(full_dataframe))
# full_dataframe.head() # Just show the first 5 rows (the head) of our dataframe 

print('you can investigate the following electrophysiolgoy features:')

w_vars = widgets.SelectMultiple(
    options=list(dataframe.columns),
    rows=10,disabled=False
)
display(w_vars)
# print('Dropdowns generated at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

NameError: name 'ctc' is not defined

Let's first look at the speed of the trough, and the ratio between the upstroke and downstroke of the action potential:
- **Action potential fast trough** (<code>fast_trough_v_long_square</code>): Minimum value of the membrane potential in the interval lasting 5 ms after the peak.
- **Upstroke/downstroke ratio** (<code>upstroke_downstroke_ratio_long_square</code>)</b>: The ratio between the absolute values of the action potential peak upstroke and the action potential peak downstroke.</div>

Select these two variables from the dropdown above.

In [ ]:
#@markdown **TASK:** Run this cell to create a scatterplot of the two selected parameters
first_selection = dataframe[w_vars.value[0]]
second_selection = dataframe[w_vars.value[1]]


fig = go.Figure()
# make_subplots(rows=2,cols=1,shared_xaxes=True,
#                     vertical_spacing=0.2)
fig.add_trace(go.Scatter(x = first_selection, y = second_selection,
                         opacity=0.5,
                         line_color='black',mode='markers'))

fig.update_layout(xaxis_title=w_vars.value[0], 
                  yaxis_title=w_vars.value[1],
                  width=450, height=450)
fig.show()

It looks like there may be roughly two clusters in the data when comparing the upstroke-downstroke ratio and the fast trough voltage. Maybe the clusters relate to whether the cells are spiny or aspiny. Cells with spiny dendrites are typically excitatory cells. Cells with aspiny dendrites  are typically inhibitory cells. We can query the database for dendrite type and use that information to split up the two sets to see.

>The cell below will dig up the dendrite type of these cells and add that to our dataframe. Then, it'll create our same scatterplot, where each dot is colored by dendrite type. All you need to do is run the cell!

In [ ]:
#@markdown **TASK:** Run this cell to dig up the dendrite types
#@markdown and create the same scatterplot, but where each dot is colored by dendrite type. 
#@markdown 
################################

# Create a dataframe for spiny cells, and a dataframe for aspiny cells
spiny_df = full_dataframe[full_dataframe['dendrite_type'] == 'spiny']
aspiny_df = full_dataframe[full_dataframe['dendrite_type'] == 'aspiny']

first_selection = spiny_df[w_vars.value[0]]
second_selection = spiny_df[w_vars.value[1]]

fig = go.Figure()
# make_subplots(rows=2,cols=1,shared_xaxes=True,
#                     vertical_spacing=0.2)
first_selection = spiny_df[w_vars.value[0]]
second_selection = spiny_df[w_vars.value[1]]
fig.add_trace(go.Scatter(x = first_selection, y = second_selection,
                         opacity=0.5,name='spiny',
                         line_color='purple',mode='markers'))
first_selection = aspiny_df[w_vars.value[0]]
second_selection = aspiny_df[w_vars.value[1]]
fig.add_trace(go.Scatter(x = first_selection, y = second_selection,
                         opacity=0.5,name='aspiny',
                         line_color='orange',mode='markers'))

fig.update_layout(xaxis_title=w_vars.value[0], 
                  yaxis_title=w_vars.value[1],
                  width=450, height=450)
fig.show()

print('you plotted the following electrophysiolgoy features:')

w_selected = widgets.Select(
    options=w_vars.value,
    disabled=False
)
display(w_selected)
# print('Dropdowns generated at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

What do you think? Did dendrite type correlate with the clusters based on these two parameters?

## Compare Action Potential Waveforms
Let's take a closer look at individual cells of each type to see what these features actually mean for the action potential waveform. Choose one of the two features you just plotted from the dropdown below the figure in step 2.3. Get the value for that feature for each of two cells in the scatterplot. 

In [ ]:
#@markdown **TASK:** Specify the 
#@markdown parameter value for two cells. Then run this cell.

value_1 =  None#@param {type:"number"} 
value_2 = None #@param {type:"number"} 
cell1_id = dataframe[w_selected.value].sub(value_1).abs().idxmin()
cell2_id = dataframe[w_selected.value].sub(value_2).abs().idxmin()



win = 0.01
cell_data = ctc.get_ephys_data(cell1_id)
# spkt = cell_data.get_spike_times(rheobase_sweep_cell1)
print('Cell #1 ID with %s = %.4f is: %i' %(w_selected.value,value_1,cell1_id))
Ival = cell_data.get_sweep_metadata(dataframe.loc[cell1_id].rheobase_sweep_number)['aibs_stimulus_amplitude_pa']
print('rheobase current = %i pA' %Ival)
print('')

sweep = cell_data.get_sweep(dataframe.loc[cell1_id].rheobase_sweep_number)
spkt = cell_data.get_spike_times(dataframe.loc[cell1_id].rheobase_sweep_number)[0]
spk_1 = sweep['response'][int((spkt-win)*sweep['sampling_rate']): int((spkt+win)*sweep['sampling_rate'])]*1e3
time_1 = np.linspace(-win, win, len(spk_1))*1e3


cell_data = ctc.get_ephys_data(cell2_id)
print('Cell #2 ID with %s = %.4f is: %i' %(w_selected.value,value_2,cell2_id))
Ival = cell_data.get_sweep_metadata(dataframe.loc[cell2_id].rheobase_sweep_number)['aibs_stimulus_amplitude_pa']
print('rheobase current = %i pA' %Ival)

# spkt = cell_data.get_spike_times(rheobase_sweep_cell1)
sweep = cell_data.get_sweep(dataframe.loc[cell2_id].rheobase_sweep_number)
spkt = cell_data.get_spike_times(dataframe.loc[cell2_id].rheobase_sweep_number)[0]
spk_2 = sweep['response'][int((spkt-win)*sweep['sampling_rate']): int((spkt+win)*sweep['sampling_rate'])]*1e3
time_2 = np.linspace(-win, win, len(spk_2))*1e3

fig = make_subplots(rows=2,cols=1,shared_xaxes=True) #go.Figure()
fig.add_trace(go.Scatter(x = time_1, y = spk_1,
                         name='cell 1',
                         line_color='black',mode='lines'),row=1,col=1)
fig.add_trace(go.Scatter(x = time_2, y = spk_2,
                         name='cell 2',
                         line_color='black',mode='lines'),row=2,col=1)
fig.update_layout(yaxis_title='mV cell 1', 
                  yaxis2_title='mV cell 2',
                  xaxis2_title='time (ms)',
                  showlegend=False,
                  width=400, height=500)
fig.show()


# Create a dataframe for spiny cells, and a dataframe for aspiny cells
spiny_df = full_dataframe[full_dataframe['dendrite_type'] == 'spiny']
aspiny_df = full_dataframe[full_dataframe['dendrite_type'] == 'aspiny']

first_selection = spiny_df[w_vars.value[0]]
second_selection = spiny_df[w_vars.value[1]]

fig = go.Figure()
# make_subplots(rows=2,cols=1,shared_xaxes=True,
#                     vertical_spacing=0.2)
first_selection = spiny_df[w_vars.value[0]]
second_selection = spiny_df[w_vars.value[1]]
fig.add_trace(go.Scatter(x = first_selection, y = second_selection,
                         opacity=0.5,name='spiny',
                         line_color='purple',mode='markers'))
first_selection = aspiny_df[w_vars.value[0]]
second_selection = aspiny_df[w_vars.value[1]]
fig.add_trace(go.Scatter(x = first_selection, y = second_selection,
                         opacity=0.5,name='aspiny',
                         line_color='orange',mode='markers'))
fig.add_trace(go.Scatter(
    x = [dataframe.loc[cell1_id][w_vars.value[0]]],
    y = [dataframe.loc[cell1_id][w_vars.value[1]]],
    mode='markers',marker_symbol='x',name='cell 1',
    marker_size=10,marker_color='black'
))

fig.add_trace(go.Scatter(
    x = [dataframe.loc[cell2_id][w_vars.value[0]]],
    y = [dataframe.loc[cell2_id][w_vars.value[1]]],
    mode='markers',marker_symbol='star',name='cell 2',
    marker_size=10,marker_color='black'
))

fig.update_layout(xaxis_title=w_vars.value[0], 
                  yaxis_title=w_vars.value[1],
                  width=450, height=450)
fig.show()


A difference in even just one intrinsic neural property can mean that the action potential shape differs dramatically. 

**TASK:** Explore the relationship among different parameters. Change your selections in the dropdown menu in step 2.1 then re-run steps 2.2-2.4 to examine different features. You can now ignore dendrite type when making your selections for step 2.4 - instead, select cells at each extreme of one of your parameter ranges to compare. 


## Compare cortical layers.


In [ ]:
#@markdown **TASK:** Activate the dropdown menus of 
#@markdown cortical layers and intrinsic ephys features

w_layer = widgets.SelectMultiple(
    options=full_dataframe['structure_layer_name'].unique(),
    rows=10,disabled=False
)

print('You can investigate comparisons')
print('among the following layers:')
display(w_layer)
print('')
w_vars_2 = widgets.Select(
    options=list(dataframe.columns),
    rows=10,disabled=False
)
print('For the following ephys features:')

display(w_vars_2)
# print('Dropdowns generated at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

In [ ]:
#@markdown **TASK:** Run this cell to 
#@markdown to plot a histogram of the selected feature 
#@markdown for each selected layer.
fig = go.Figure()
# make_subplots(rows=2,cols=1,shared_xaxes=True,
#                     vertical_spacing=0.2)

# fig.add_trace(go.Histogram(x=x0))
# fig.add_trace(go.Histogram(x=x1))
for col in w_layer.value:
  mask = full_dataframe['structure_layer_name']==col
  ephys = dataframe.loc[mask,w_vars_2.value]
  medval = np.nanmedian(ephys)
  fig.add_trace(go.Scatter(
      x=[medval],y=[0],marker_size=10,
      name='layer ' +col + ' median', marker_symbol='triangle-up'
  ))
  fig.add_trace(go.Histogram(x = ephys, 
                         opacity=1,name='layer' + col,
                         histnorm='probability'))

fig.update_layout(xaxis_title=w_vars_2.value, 
                  yaxis_title='probability',
                  width=600, height=450)
fig.show()

print('you plotted the following layers:')

w_selected_2 = widgets.SelectMultiple(
    options=w_layer.value,
    disabled=False
)
display(w_selected_2)

You might be asking yourself some questions as you examine your comparisons, such as: 

- Do differences among cortical layers in this feature correspond to differences in action potential physiology? 

- Do cells from different cortical layers differ in their action potential physiology even though they are similar in the above parameter?

- What makes more of a difference in determining a neuron's action potential physiology - the layer in which it is found or the value of a particular physiological feature?

- etc...

To answer these questions, select, from the cortical layers in the dropdown above, whichever you want to compare spike waveforms for. Then run the cell below.

In [ ]:
#@markdown **TASK:** 
#@markdown Run this cell to plot the action potential 
#@markdown for a 'canonical' (median value) cell from each specified layer
#@markdown for the selected parameter.
# parameter_value =  0.08#@param {type:"number"} 
# value_2 =  1.93#@param {type:"number"} 

fig = go.Figure()
for col in w_selected_2.value:
  mask = full_dataframe['structure_layer_name']==col
  parameter_value = np.nanmedian(dataframe.loc[mask,w_vars_2.value])
  cell_id = dataframe.loc[mask,w_vars_2.value].sub(parameter_value).abs().idxmin()

  win = 0.01
  cell_data = ctc.get_ephys_data(cell_id);
  # spkt = cell_data.get_spike_times(rheobase_sweep_cell1)
  print('')
  print('Layer %s cell with %s = %.4f is: %i' %(col,w_vars_2.value,parameter_value,cell_id))
  Ival = cell_data.get_sweep_metadata(dataframe.loc[cell_id].rheobase_sweep_number)['aibs_stimulus_amplitude_pa']
  print('rheobase current = %i pA' %Ival)
  print('')

  sweep = cell_data.get_sweep(dataframe.loc[cell_id].rheobase_sweep_number)
  spkt = cell_data.get_spike_times(dataframe.loc[cell_id].rheobase_sweep_number)[0]
  spk = sweep['response'][int((spkt-win)*sweep['sampling_rate']): int((spkt+win)*sweep['sampling_rate'])]*1e3
  time = np.linspace(-win, win, len(spk))*1e3

  fig.add_trace(go.Scatter(x = time, y = spk,
                         name='layer ' + col,mode='lines'))

fig.update_layout(yaxis_title='mV', 
                  xaxis_title='time (ms)',
                  width=600, height=450)
fig.show()



### Choose a different feature to compare among the same cortical layers. 

Can you find cortical layers that are distinct in a particular electrophysiological feature and/or features that are distinct among particular layers?

Here are some pre-computed features you might consider comparing (you can find a complete glossary [here](https://docs.google.com/document/d/1YGLwkMTebwrXd_1E817LFbztMjSTCWh83Mlp3_3ZMEo/edit?usp=sharing)):

- <b>Tau (<code>tau</code>)</b>: time constant of the membrane in milliseconds
- <b>Adapation ratio (<code>adaptation</code>)</b>: The rate at which firing speeds up or slows down during a stimulus<br>
- <b>Average ISI (<code>avg_isi</code>)</b>: The mean value of all interspike intervals in a sweep<br>
- **Slope of f/I curve** (<code>f_i_curve_slope</code>)</b>: slope of the curve between firing rate (f) and current injected<br>
- **Input Resistance** (<code>input_resistance_mohm</code>)</b>: The input resistance of the cell, in megaohms.<br>
- **Voltage of after-hyperpolarization** (<code>trough_v_short_square</code>)</b>: minimum value of the membrane potential during the after-hyperpolarization



## Neuron diversity among layers

Here, we are visualizing action potential physiology of the canonical (median) neuron for each layer based on a electrophysiological parameter. However, there is cell-type diversity within layers as well. You can explore this diversity extensively in an orderly and efficient manner by writing custom code to query the Allen Database. However, even without any coding experience, you can begin explore this diversity by:
1. going to the ["Cell Feature Search"](https://celltypes.brain-map.org/data) page at the Allen Institute
2. selecting specific layers and/or brain regions
3. sorting the neurons in that layer in ascending/descending order based on a specific parameter 
4. browsing the thumbnail images
5. clicking on individual thumbnails to see more detail  

# Plot sweeps from individual neurons

Using this cell, you can plot specific sweeps from any cell in the database. 

In [ ]:
#@markdown **TASK:** Enter the information needed in this form 
#@markdown and then run the code cell to plot the sweep from that neuron.
#@markdown >Note: You can change the figure size (re-run the code cell after changing)
cell_id =  None #@param {type:"number"} 
sweep_number = None #@param {type:"number"} 
figure_width = 500 #@param {type:"number"} 
figure_height = 450 #@param {type:"number"} 

cell_data = ctc.get_ephys_data(cell1_id)
sweep = cell_data.get_sweep(sweep_number)

fig = make_subplots(rows=2, cols=1, 
                    row_heights=[0.7, 0.3]
                    ,shared_xaxes=True) # go.Figure()

voltage = sweep['response']*1e3
new_fs = 5000
step = int(sweep['sampling_rate']/new_fs)
voltage_rs = voltage[0::step]
time = np.linspace(0,len(voltage)/sweep['sampling_rate'],len(voltage_rs))

fig.add_trace(go.Scatter(x = time, y = voltage_rs,
                         line_color='black',mode='lines'),
              row=1,col=1)

current = sweep['stimulus']*1e12
new_fs = 500
step = int(sweep['sampling_rate']/new_fs)
current_rs = current[0::step]
time = np.linspace(0,len(current)/sweep['sampling_rate'],len(current_rs))

fig.add_trace(go.Scatter(x = time, y = current_rs,
                         line_color='black',mode='lines'),
              row=2,col=1)

fig.update_layout(yaxis_title='mV', 
                  yaxis2_title='pA',
                  xaxis2_title='time (s)',
                  showlegend=False,
                  width=figure_width, height=figure_height)
fig.show()

<hr>

# That's it for today -- great work!

In [ ]:
#@markdown RUN this cell to celebrate
from IPython.display import HTML
print('Nice work!')
HTML('<img src="https://media.giphy.com/media/xUOwGhOrYP0jP6iAy4/giphy.gif">')

-----------

# Technical notes & credits

See [Technical Notes](#technical) for more information about working with the AllenSDK.

This notebook demonstrates most of the features of the AllenSDK that help manipulate data in the Cell Types Database.  The main entry point will be through the `CellTypesCache` class. `CellTypesCache` is responsible for downloading Cell Types Database data to a standard directory structure on your hard drive.  If you use this class, you will not have to keep track of where your data lives, other than a root directory.

Much more information can be found in the <a href="http://help.brain-map.org/download/attachments/8323525/CellTypes_Ephys_Overview.pdf">Allen Brain Atlas whitepaper</a> as well as in their <a href="http://alleninstitute.github.io/AllenSDK/cell_types.html">GitHub documentation</a>.

This file modified from <a href='https://alleninstitute.github.io/AllenSDK/_static/examples/nb/cell_types.html'>this</a> notebook.

You can always Google questions you have!)